<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/GeneticAlgorithm_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import numba as nb
from plotly.subplots import make_subplots
np.set_printoptions(linewidth=200,formatter={'float':lambda x: f'{x:4.2f}'})

In [ ]:
rng=np.random.RandomState(123)

In [ ]:
N=100
coords=rng.uniform(0,10, size=(N,2))

In [ ]:
dist=np.sqrt(np.sum((coords[:,None,:]-coords[None,:,:])**2,axis=2))
np.fill_diagonal(dist,100)

In [ ]:
def plotter(seq_plot, dists, nr, nc):
    fig=make_subplots(rows = nr, cols=nc)
    seq = np.r_[seq_plot, np.atleast_2d(seq_plot[0,:])]
    for i in range(seq.shape[1]):
        fig.add_scatter(x=coords[seq[:,i],0], y=coords[seq[:,i],1], row=i//nc+1, col=i%nc +1)
        fig.add_annotation(x=10,y=12,text=f'{dists[i]:.2f}', xanchor='right',showarrow=False, row=i//nc+1,col=i%nc + 1)
    fig.update_layout(width=200*nc, height=200*nr,template='plotly_dark', showlegend=False)
    fig.show()

In [ ]:
def calcdist(seq):
    return np.sum(dist[seq[1:,:],seq[:-1,:]],axis=0) + dist[seq[0,:],seq[-1,:]]

In [ ]:
def topn(seq_in,n,unique=False):
    seq=seq_in
    if unique:
        seq=np.unique(seq_in,axis=1)
    dists=calcdist(seq)
    topn_idx=np.argsort(dists)[:n]
    return seq[:,topn_idx], dists[topn_idx]

In [ ]:
@nb.njit
def mutate(seq):
    N,popsize= seq.shape
    for idx in range(popsize):
        n=np.random.randint(N)
        cur=seq[n,idx]
        close=np.where(seq[:,idx]==(np.argsort(dist[cur,:]))[np.random.randint(5)])[0][0]
        a,b=min(n,close),max(n,close)
        if b==N-1:
            seq[:,idx]=np.concatenate((seq[:a,idx],seq[a:b+1,idx][::-1]))
        else:
            seq[:,idx]=np.concatenate((seq[:a,idx],seq[a:b,idx][::-1],seq[b:,idx]))

In [ ]:
@nb.njit
def isin(v,a):
    for i in a:
        if v==i:
            return True
    return False

@nb.njit
def swap(p1,p2,c1,c2):
    o=np.zeros_like(p1)
    N=p1.size
    swap=np.concatenate( (p2[c2:],p2[:c2]))
    filtered=[]
    for x in swap:
        if not(isin(x,p1[c1:c2])):
            filtered.append(x)
    o[c1:c2]=p1[c1:c2]
    o[c2:N]=filtered[:(N-c2)]
    o[:c1]=filtered[(N-c2):]
    return o

@nb.njit
def mate2p(p1,p2):
    N=p1.size
    c1=np.random.randint(N//4,N//2)
    c2=np.random.randint(N//2,3*N//4)

    o1=swap(p1,p2,c1,c2)
    o2=swap(p2,p1,c1,c2)
    return o1,o2

@nb.njit
def mate(s):
    seq=s[:,np.random.permutation(np.arange(s.shape[1]))]
    offspring=np.zeros_like(seq)
    for i,(p1,p2) in enumerate(zip(seq[:,::2].T,seq[:,1::2].T)):
        offspring[:,i*2],offspring[:,i*2+1]=mate2p(p1,p2)
    return offspring      

In [ ]:
popsize = 512
seq=np.zeros((N,popsize), dtype=int)

for popidx in range(popsize):

    remain = list(range(N))

    cur = rng.randint(N)
    seq[0,popidx]=cur
    remain.remove(cur)
    for i in range(1,N-1):
        best2= np.argsort(dist[cur,remain])[:2]
        best=remain[best2[rng.choice([0,1],p=(0.95,0.05))]  ]
        seq[i,popidx]=best
        cur=best
        remain.remove(best)
    seq[-1,popidx]=remain[0]


In [ ]:
N_gen=1000
for _ in range(N_gen):
    better_seq, better_dists =topn(seq,popsize//2,unique=True)
    children=mate(better_seq)

    mutate_idx = rng.choice(range(better_seq.shape[1]),size=int(0.9*better_seq.shape[1]),replace=False)
    mutated_better_seq=better_seq[:,mutate_idx]
    mutate(mutated_better_seq)

    seq=np.concatenate([better_seq, mutated_better_seq,children],axis=1)

In [ ]:
bestseq,bestdist=topn(seq,28,unique=True)
plotter(bestseq,bestdist,nr=4,nc=7)